In [12]:
#%load_ext autoreload
#%autoreload all

# Programming

## Setup

Run the below code in a cell to install the dependencies.

```python
%pip install --quiet tqdm pandas
%pip install --quiet --extra-index-url https://test.pypi.org/simple/ git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-OOEP
# NOTE local repo
#%pip install --extra-index-url https://test.pypi.org/simple/ ../../..
%pip install git+https://github.com/NTU-CCA-HVAC-OPTIM-a842a748/EnergyPlus-Datasets.git
```

## Importing ~~`pyenergyplus`~~ `ooep`

In [15]:
import energyplus.ooep as ooep
import energyplus.ooep.ems

## `ooep` - EnergyPlus API, Object-Oriented

<table style="text-align: center">
    <caption>Architecture Overview (*NIX)</caption>
    <tr>
        <th>EnergyPlus<br>OO Abstraction Layer</th>
        <th rowspan="2">&rlhar;</th>
        <th colspan="3">EnergyPlus<br>State Machine</th>
    </tr>
    <tr>
        <td>
            API<br>
            <code>ooep</code>
        </td>
        <td>
            API<br>
            (<code>pyenergyplus</code>)<br>
            (<code>libenergyplusapi.so</code>)
        </td>
        <td>ABI<br>&rlhar;</td>
        <td>Kernel<br>(<code>energyplus</code>)</td>
    </tr>
</table>

In [16]:
from energyplus.dataset.basic import dataset as epds

list(epds.models.glob('*.idf'))[:10]

[PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/5ZoneWaterCooled_Baseboard.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/PipeHeatTransfer_Schedule.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/5ZoneNightVent2.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/UserDefinedRoomAirPatterns.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/DualDuctVarVolDamper.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/PythonPluginCurveOverride_PackagedTerminalHeatPump.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-packages/energyplus/dataset/basic/assets/models/ZoneWSHP_wDOAS.idf'),
 PosixPath('/home/sysadmin/.local/lib/python3.11/site-

### `ooep` in Action

#### Setup

- (Context/State Management) Enter `ooep.ems` environment...
  Don't forget to `__close__` it when done \
  OR better yet, just use `python`'s `with`-statement:
  ```python
    import ooep.ems
    with ooep.ems.Environment() as e:
        # NOTE do whatever you need to do with `e`
        ...
  ```

In [17]:
env = ooep.ems.Environment().__enter__()

- 🎉

In [18]:
env('--help', verbose=True)


PythonLinkage: This version of EnergyPlus not linked to Python library.
Built on Platform: Ubuntu22.04_x86_64

Usage: energyplus [OPTIONS] [input_file]

Positionals:
  input_file TEXT:FILE        Input file (default: in.idf in current directory)

Options:
  -h,--help                   Print this help message and exit
  -v,--version                Display program version information and exit
  -a,--annual Excludes: --design-day
                              Force annual simulation
  -D,--design-day Excludes: --annual
                              Force design-day-only simulation
  -d,--output-directory DIR   Output directory path (default: current directory)
  -i,--idd IDD                Input data dictionary path (default: Energy+.idd in executable directory)
  -m,--epmacro                Run EPMacro prior to simulation
  -p,--output-prefix PRE      Prefix for output file names (default: eplus)
  -r,--readvars               Run ReadVarsESO after simulation
  -c,--convert              

0

  Output IDF->epJSON or epJSON->IDF, dependent on input file type
  --convert-only              Only convert IDF->epJSON or epJSON->IDF, dependent on input file type. No simulation
  -s,--output-suffix SUFFIX   Suffix style for output file names (default: L)
                                 L: Legacy (e.g., eplustbl.csv)
                                 C: Capital (e.g., eplusTable.csv)
                                 D: Dash (e.g., eplus-table.csv)
  -j,--jobs N                 Multi-thread with N threads; 1 thread with no arg. (Currently only for G-Function generation)
  -w,--weather EPW            Weather file path (default: in.epw in current directory)
  -x,--expandobjects          Run ExpandObjects prior to simulation


Example: energyplus -w weather.epw -r input.idf


In [19]:
env = env.__enter__()

env(
    # TODO
    '--design-day',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{epds.weathers}/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    #f'{epds.models}/CoolingTower_VariableSpeed_MultiCell.idf'
    f'{epds.models}/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

0

#### Variables

In [20]:
env.specs.variables

,variable_name,variable_key
0,Site Outdoor Air Drybulb Temperature,ENVIRONMENT
1,Site Outdoor Air Dewpoint Temperature,ENVIRONMENT
2,Site Outdoor Air Wetbulb Temperature,ENVIRONMENT
3,Site Wind Speed,ENVIRONMENT
4,Site Sky Temperature,ENVIRONMENT
...,...,...
2095,Zone Mechanical Ventilation No Load Heat Addit...,DATACENTER_BASEMENT_ZN_6
2096,Zone Mechanical Ventilation Heating Load Incre...,DATACENTER_BASEMENT_ZN_6
2097,Zone Mechanical Ventilation Heating Load Incre...,DATACENTER_BASEMENT_ZN_6
2098,Zone Mechanical Ventilation Heating Load Decre...,DATACENTER_BASEMENT_ZN_6


- Variable keys

In [21]:
env.specs.variables['variable_key'].unique()

array(['ENVIRONMENT', 'BASEMENT', 'CORE_BOTTOM', 'CORE_MID', 'CORE_TOP',
       'PERIMETER_BOT_ZN_3', 'PERIMETER_BOT_ZN_2', 'PERIMETER_BOT_ZN_1',
       'PERIMETER_BOT_ZN_4', 'PERIMETER_MID_ZN_3', 'PERIMETER_MID_ZN_2',
       'PERIMETER_MID_ZN_1', 'PERIMETER_MID_ZN_4', 'PERIMETER_TOP_ZN_3',
       'PERIMETER_TOP_ZN_2', 'PERIMETER_TOP_ZN_1', 'PERIMETER_TOP_ZN_4',
       'GROUNDFLOOR_PLENUM', 'MIDFLOOR_PLENUM', 'TOPFLOOR_PLENUM',
       'DATACENTER_BOT_ZN_6', 'DATACENTER_MID_ZN_6',
       'DATACENTER_TOP_ZN_6', 'DATACENTER_BASEMENT_ZN_6',
       'BASEMENT_LIGHTS', 'CORE_BOTTOM_LIGHTS', 'CORE_MID_LIGHTS',
       'CORE_TOP_LIGHTS', 'PERIMETER_BOT_ZN_3_LIGHTS',
       'PERIMETER_BOT_ZN_2_LIGHTS', 'PERIMETER_BOT_ZN_1_LIGHTS',
       'PERIMETER_BOT_ZN_4_LIGHTS', 'PERIMETER_MID_ZN_3_LIGHTS',
       'PERIMETER_MID_ZN_2_LIGHTS', 'PERIMETER_MID_ZN_1_LIGHTS',
       'PERIMETER_MID_ZN_4_LIGHTS', 'PERIMETER_TOP_ZN_3_LIGHTS',
       'PERIMETER_TOP_ZN_2_LIGHTS', 'PERIMETER_TOP_ZN_1_LIGHTS',
       'PE

In [22]:
env.specs.variables[
    env.specs.variables['variable_key']
        .str
        .contains('CORE_MID')
]

,variable_name,variable_key
17,Zone Total Internal Latent Gain Energy,CORE_MID
18,Zone Total Internal Latent Gain Rate,CORE_MID
65,People Occupant Count,CORE_MID
66,People Air Temperature,CORE_MID
67,People Air Relative Humidity,CORE_MID
111,Zone People Occupant Count,CORE_MID
112,Zone People Total Heating Energy,CORE_MID
141,Lights Electricity Energy,CORE_MID_LIGHTS
165,Zone Lights Electricity Rate,CORE_MID
166,Zone Lights Electricity Energy,CORE_MID


#### Internal Variables

In [23]:
env.specs.internal_variables

,variable_type,variable_key
0,Plant Design Volume Flow Rate,SHWSYS1
1,Plant Design Volume Flow Rate,HEATSYS1
2,Plant Design Volume Flow Rate,COOLSYS1_SUPPLY
3,Plant Design Volume Flow Rate,COOLSYS1_DEMAND
4,Plant Design Volume Flow Rate,TOWERWATERSYS
...,...,...
743,Pump Maximum Mass Flow Rate,SHWSYS1 PUMP
744,Pump Maximum Mass Flow Rate,COOLSYS1 CHILLER1 PUMP
745,Pump Maximum Mass Flow Rate,COOLSYS1 CHILLER2 PUMP
746,Pump Maximum Mass Flow Rate,PLANT CIRC PUMP


#### Meters

In [24]:
env.specs.meters

,meter_name
0,Electricity:Facility
1,Electricity:Building
2,Electricity:Zone:BASEMENT
3,Electricity:SpaceType:GENERAL
4,InteriorLights:Electricity
...,...
314,General:Pumps:Electricity
315,Carbon Equivalent:Facility
316,CarbonEquivalentEmissions:Carbon Equivalent
317,WIRED_LTG_ELECTRICITY


#### Actuators

In [26]:
env.specs.actuators

,component_type,control_type,actuator_key
0,Weather Data,Outdoor Dry Bulb,Environment
1,Weather Data,Outdoor Dew Point,Environment
2,Weather Data,Outdoor Relative Humidity,Environment
3,Weather Data,Diffuse Solar,Environment
4,Weather Data,Direct Solar,Environment
...,...,...,...
5085,AirLoopHVAC,Availability Status,VAV_TOP WITH REHEAT
5086,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER BASEMENT
5087,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER BOT
5088,AirLoopHVAC,Availability Status,AIRLOOP DATACENTER MID


- Component types of the actuators...

In [27]:
env.specs.actuators['component_type'].unique()

array(['Weather Data', 'Schedule:Compact', 'Schedule:Constant',
       'Material', 'People', 'Lights', 'ElectricEquipment', 'Surface',
       'Zone', 'Zone Infiltration', 'Plant Loop Overall',
       'Supply Side Half Loop', 'Demand Side Half Loop',
       'Demand Side Branch', 'Plant Component Pipe:Adiabatic',
       'Plant Component WaterUse:Connections', 'Supply Side Branch',
       'Plant Component Pump:ConstantSpeed',
       'Plant Component WaterHeater:Mixed',
       'Plant Component Coil:Heating:Water',
       'Plant Component Pump:VariableSpeed',
       'Plant Component Boiler:HotWater',
       'Plant Component HeatExchanger:FluidToFluid',
       'Plant Component Chiller:Electric:ASHRAE205',
       'Plant Component Coil:Cooling:Water',
       'Plant Component Coil:Cooling:WaterToAirHeatPump:EquationFit',
       'Plant Component Coil:Heating:WaterToAirHeatPump:EquationFit',
       'Plant Component FluidCooler:TwoSpeed',
       'Plant Component HeaderedPumps:VariableSpeed',
     

In [28]:
env.specs.actuators[
    env.specs.actuators['component_type'] 
        == 'Plant Component Chiller:Electric:ASHRAE205'
]

,component_type,control_type,actuator_key
1424,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER1
1427,Plant Component Chiller:Electric:ASHRAE205,On/Off Supervisory,COOLSYS1 CHILLER2


In [29]:
env.specs.actuators[
    env.specs.actuators['component_type'] 
        == 'Plant Component CoolingTower:VariableSpeed'
]

,component_type,control_type,actuator_key
1504,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 1
1506,Plant Component CoolingTower:VariableSpeed,On/Off Supervisory,TOWERWATERSYS COOLTOWER 2


In [30]:
env.specs.actuators[
    env.specs.actuators['component_type'].isin(
        ('Zone Temperature Control', 'Zone Humidity Control')
    )
]

,component_type,control_type,actuator_key
1631,Zone Temperature Control,Heating Setpoint,BASEMENT
1632,Zone Temperature Control,Cooling Setpoint,BASEMENT
1633,Zone Temperature Control,Heating Setpoint,CORE_BOTTOM
1634,Zone Temperature Control,Cooling Setpoint,CORE_BOTTOM
1635,Zone Temperature Control,Heating Setpoint,CORE_MID
1636,Zone Temperature Control,Cooling Setpoint,CORE_MID
1637,Zone Temperature Control,Heating Setpoint,CORE_TOP
1638,Zone Temperature Control,Cooling Setpoint,CORE_TOP
1639,Zone Temperature Control,Heating Setpoint,PERIMETER_BOT_ZN_3
1640,Zone Temperature Control,Cooling Setpoint,PERIMETER_BOT_ZN_3


#### Events

In [31]:
env.specs.events

,event_name
0,after_component_input
1,after_new_environment_warmup_complete
2,after_predictor_after_hvac_managers
3,after_predictor_before_hvac_managers
4,begin_new_environment
5,begin_system_timestep_before_predictor
6,begin_zone_timestep_after_init_heat_balance
7,begin_zone_timestep_before_init_heat_balance
8,begin_zone_timestep_before_set_current_weather
9,end_system_sizing


### Simulation

In [32]:
env = env.__enter__()

In [33]:
_ = env(
    # TODO
    '--design-day',
    #'--annual',
    '--output-directory', 'build/demo-eplus',
    '--weather', f'{epds.weathers}/USA_FL_Tampa.Intl.AP.722110_TMY3.epw',
    f'{epds.models}/ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
)

In [ ]:
env.__exit__()